In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]

In [3]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 1s 0us/step


In [4]:
for layer in resnet.layers:
    layer.trainable = False

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [11]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [12]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [13]:
folders = glob('/content/gdrive/My Drive/Kaggle/chest_xray/train/*')

In [15]:
x = Flatten()(resnet.output)

In [16]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [17]:
# view the structure of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [18]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [21]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/gdrive/My Drive/Kaggle/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [22]:
test_set = test_datagen.flow_from_directory('/content/gdrive/My Drive/Kaggle/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [24]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 950s 6s/step - loss: 0.3378 - accuracy: 0.8727 - val_loss: 0.4821 - val_accuracy: 0.8045
Epoch 2/5
163/163 [==============================] - 951s 6s/step - loss: 0.2926 - accuracy: 0.8892 - val_loss: 0.3004 - val_accuracy: 0.8750
Epoch 3/5
163/163 [==============================] - 947s 6s/step - loss: 0.2387 - accuracy: 0.9059 - val_loss: 0.3957 - val_accuracy: 0.8317
Epoch 4/5
163/163 [==============================] - 950s 6s/step - loss: 0.2899 - accuracy: 0.8955 - val_loss: 1.4610 - val_accuracy: 0.6603
Epoch 5/5
163/163 [==============================] - 944s 6s/step - loss: 0.3111 - accuracy: 0.8850 - val_loss: 1.4271 - val_accuracy: 0.6715


In [28]:
from tensorflow.keras.models import load_model

model.save('m_resnet50.h5')

In [29]:
y_pred = model.predict(test_set)

In [30]:
y_pred

array([[5.9548497e-06, 9.9999404e-01],
       [1.9371937e-04, 9.9980634e-01],
       [1.9920094e-06, 9.9999797e-01],
       ...,
       [3.3684000e-06, 9.9999666e-01],
       [8.2051205e-03, 9.9179488e-01],
       [1.3216064e-04, 9.9986780e-01]], dtype=float32)

In [31]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [33]:
model=load_model('m_resnet50.h5')

In [46]:
img=image.load_img('/content/gdrive/My Drive/Kaggle/chest_xray/test/PNEUMONIA/person1_virus_6.jpeg',target_size=(224,224))

In [47]:
x=image.img_to_array(img)
x

array([[[220., 220., 220.],
        [215., 215., 215.],
        [211., 211., 211.],
        ...,
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.],
        [ 12.,  12.,  12.]],

       [[218., 218., 218.],
        [216., 216., 216.],
        [212., 212., 212.],
        ...,
        [ 18.,  18.,  18.],
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.]],

       [[215., 215., 215.],
        [216., 216., 216.],
        [209., 209., 209.],
        ...,
        [ 34.,  34.,  34.],
        [ 18.,  18.,  18.],
        [ 11.,  11.,  11.]],

       ...,

       [[ 23.,  23.,  23.],
        [ 23.,  23.,  23.],
        [ 23.,  23.,  23.],
        ...,
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.]],

       [[ 23.,  23.,  23.],
        [ 23.,  23.,  23.],
        [ 23.,  23.,  23.],
        ...,
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.],
        [ 13.,  13.,  13.]],

       [[ 23.,  23.,  23.],
        [ 23.,  23.,  23.],
        [ 23.,  

In [48]:
x.shape


(224, 224, 3)

In [49]:
x=x/255

In [50]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [51]:
model.predict(img_data)

array([[1.9682889e-04, 9.9980325e-01]], dtype=float32)

In [52]:
a=np.argmax(model.predict(img_data), axis=1)

In [55]:
a==1

array([ True])